In [1]:
import numpy as np

class AHRS:
    def __init__(self, sample_period=1/256, quaternion=None, kp=2, ki=0, kp_init=200, init_period=5):
        self.sample_period = sample_period
        self.quaternion = quaternion if quaternion is not None else np.array([1, 0, 0, 0])
        self.Kp = kp
        self.Ki = ki
        self.KpInit = kp_init
        self.InitPeriod = init_period
        self.q = self.quaternion  # internal quaternion
        self.int_error = np.array([0, 0, 0])
        self.KpRamped = self.KpInit

    @staticmethod
    def quatern_prod(a, b):
        ab = np.zeros(4)
        ab[0] = a[0]*b[0] - a[1]*b[1] - a[2]*b[2] - a[3]*b[3]
        ab[1] = a[0]*b[1] + a[1]*b[0] + a[2]*b[3] - a[3]*b[2]
        ab[2] = a[0]*b[2] - a[1]*b[3] + a[2]*b[0] + a[3]*b[1]
        ab[3] = a[0]*b[3] + a[1]*b[2] - a[2]*b[1] + a[3]*b[0]
        return ab

    @staticmethod
    def quatern_conj(q):
        return np.array([q[0], -q[1], -q[2], -q[3]])

    def update_imu(self, gyroscope, accelerometer):
        if np.linalg.norm(accelerometer) == 0:
            print('Accelerometer magnitude is zero. Algorithm update aborted.')
            return

        # Normalize accelerometer measurement
        accelerometer /= np.linalg.norm(accelerometer)

        # Compute error between estimated and measured direction of gravity
        v = np.array([
            2*(self.q[1]*self.q[3] - self.q[0]*self.q[2]),
            2*(self.q[0]*self.q[1] + self.q[2]*self.q[3]),
            self.q[0]**2 - self.q[1]**2 - self.q[2]**2 + self.q[3]**2
        ])
        error = np.cross(v, accelerometer)

        # Compute and apply feedback terms
        ref = gyroscope - (self.Kp*error + self.Ki*self.int_error)
        p_dot = 0.5 * self.quatern_prod(self.q, np.insert(ref, 0, 0))
        self.q += p_dot * self.sample_period
        self.q /= np.linalg.norm(self.q)

        # Store conjugate
        self.quaternion = self.quatern_conj(self.q)

    def reset(self):
        self.KpRamped = self.KpInit
        self.int_error = np.array([0, 0, 0])
        self.q = np.array([1, 0, 0, 0])

In [2]:
import numpy as np

class Quaternion:
    def __init__(self, q=np.array([1, 0, 0, 0])):
        self.q = self.normalize(q)

    @staticmethod
    def normalize(q):
        return q / np.linalg.norm(q)

    @staticmethod
    def from_axis_angle(axis, angle):
        q0 = np.cos(angle / 2)
        q1 = -axis[0] * np.sin(angle / 2)
        q2 = -axis[1] * np.sin(angle / 2)
        q3 = -axis[2] * np.sin(angle / 2)
        return Quaternion(np.array([q0, q1, q2, q3]))

    @staticmethod
    def from_rotation_matrix(R):
        K = np.array([
            [1/3 * (R[0, 0] - R[1, 1] - R[2, 2]), 1/3 * (R[1, 0] + R[0, 1]), 1/3 * (R[2, 0] + R[0, 2]), 1/3 * (R[1, 2] - R[2, 1])],
            [1/3 * (R[1, 0] + R[0, 1]), 1/3 * (R[1, 1] - R[0, 0] - R[2, 2]), 1/3 * (R[2, 1] + R[1, 2]), 1/3 * (R[2, 0] - R[0, 2])],
            [1/3 * (R[2, 0] + R[0, 2]), 1/3 * (R[2, 1] + R[1, 2]), 1/3 * (R[2, 2] - R[0, 0] - R[1, 1]), 1/3 * (R[0, 1] - R[1, 0])],
            [1/3 * (R[1, 2] - R[2, 1]), 1/3 * (R[2, 0] - R[0, 2]), 1/3 * (R[0, 1] - R[1, 0]), 1/3 * (R[0, 0] + R[1, 1] + R[2, 2])]
        ])
        eigvals, eigvecs = np.linalg.eig(K)
        max_index = np.argmax(eigvals)
        q = eigvecs[:, max_index]
        return Quaternion(q)

    def to_euler(self):
        R = self.to_rotation_matrix()
        phi = np.arctan2(R[2, 1], R[2, 2])
        theta = -np.arctan(R[2, 0] / np.sqrt(1-R[2, 0]**2))
        psi = np.arctan2(R[1, 0], R[0, 0])
        return np.array([phi, theta, psi])

    def to_rotation_matrix(self):
        q = self.q
        R = np.array([
            [2*q[0]**2-1+2*q[1]**2, 2*(q[1]*q[2]+q[0]*q[3]), 2*(q[1]*q[3]-q[0]*q[2])],
            [2*(q[1]*q[2]-q[0]*q[3]), 2*q[0]**2-1+2*q[2]**2, 2*(q[2]*q[3]+q[0]*q[1])],
            [2*(q[1]*q[3]+q[0]*q[2]), 2*(q[2]*q[3]-q[0]*q[1]), 2*q[0]**2-1+2*q[3]**2]
        ])
        return R

    def conjugate(self):
        return Quaternion(np.array([self.q[0], -self.q[1], -self.q[2], -self.q[3]]))

    def product(self, other):
        a, b = self.q, other.q
        ab = np.array([
            a[0]*b[0] - a[1]*b[1] - a[2]*b[2] - a[3]*b[3],
            a[0]*b[1] + a[1]*b[0] + a[2]*b[3] - a[3]*b[2],
            a[0]*b[2] - a[1]*b[3] + a[2]*b[0] + a[3]*b[1],
            a[0]*b[3] + a[1]*b[2] - a[2]*b[1] + a[3]*b[0]
        ])
        return Quaternion(ab)

    def rotate_vector(self, v):
        qv = Quaternion(np.hstack([0, v]))
        return self.product(qv).product(self.conjugate()).q[1:]
    
    @staticmethod
    def euler_to_rot_mat(phi, theta, psi):
        R = np.array([[np.cos(psi)*np.cos(theta), -np.sin(psi)*np.cos(phi) + np.cos(psi)*np.sin(theta)*np.sin(phi), np.sin(psi)*np.sin(phi) + np.cos(psi)*np.sin(theta)*np.cos(phi)],
                      [np.sin(psi)*np.cos(theta), np.cos(psi)*np.cos(phi) + np.sin(psi)*np.sin(theta)*np.sin(phi), -np.cos(psi)*np.sin(phi) + np.sin(psi)*np.sin(theta)*np.cos(phi)],
                      [-np.sin(theta), np.cos(theta)*np.sin(phi), np.cos(theta)*np.cos(phi)]])
        return R

    @staticmethod
    def rot_mat_to_euler(R):
        phi = np.arctan2(R[2, 1], R[2, 2])
        theta = -np.arctan(R[2, 0] / np.sqrt(1-R[2, 0]**2))
        psi = np.arctan2(R[1, 0], R[0, 0])
        return np.array([phi, theta, psi])

    @staticmethod
    def rot_mat_to_quaternion(R):
        K = np.array([
            [1/3 * (R[0, 0] - R[1, 1] - R[2, 2]), 1/3 * (R[1, 0] + R[0, 1]), 1/3 * (R[2, 0] + R[0, 2]), 1/3 * (R[1, 2] - R[2, 1])],
            [1/3 * (R[1, 0] + R[0, 1]), 1/3 * (R[1, 1] - R[0, 0] - R[2, 2]), 1/3 * (R[2, 1] + R[1, 2]), 1/3 * (R[2, 0] - R[0, 2])],
            [1/3 * (R[2, 0] + R[0, 2]), 1/3 * (R[2, 1] + R[1, 2]), 1/3 * (R[2, 2] - R[0, 0] - R[1, 1]), 1/3 * (R[0, 1] - R[1, 0])],
            [1/3 * (R[1, 2] - R[2, 1]), 1/3 * (R[2, 0] - R[0, 2]), 1/3 * (R[0, 1] - R[1, 0]), 1/3 * (R[0, 0] + R[1, 1] + R[2, 2])]
        ])
        eigvals, eigvecs = np.linalg.eig(K)
        max_index = np.argmax(eigvals)
        q = eigvecs[:, max_index]
        return Quaternion(q)

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt


# 데이터셋 불러오기
file_path = 'D:\\20192230 이동섭\\IMU sensor\\3D Tracking Test\\Gait-Tracking-With-x-IMU-master\\Gait Tracking With x-IMU\\Datasets\\transformed_LoggedData_test.csv'
data = pd.read_csv(file_path)

# 센서 데이터 할당
gyrX = data['Gyroscope X (deg/s)']
gyrY = data['Gyroscope Y (deg/s)']
gyrZ = data['Gyroscope Z (deg/s)']
accX = data['Accelerometer X (g)']
accY = data['Accelerometer Y (g)']
accZ = data['Accelerometer Z (g)']

# 시간 데이터 생성
sample_period = 1/256
num_samples = data.shape[0]
time = np.linspace(0, num_samples * sample_period, num_samples)

# 데이터 프레임
start_time = 6
stop_time = 26
index_sel = (time >= start_time) & (time <= stop_time)
gyrX = gyrX[index_sel]
gyrY = gyrY[index_sel]
gyrZ = gyrZ[index_sel]
accX = accX[index_sel]
accY = accY[index_sel]
accZ = accZ[index_sel]

# 여기에 나머지 처리 코드를 추가하세요.
# 예: 정지 구간 감지, 방향 추정, 변환 가속도 계산 등

# 예를 들어, 저역통과 필터를 생성하고 데이터에 적용하는 방법:
def low_pass_filter(data, cutoff, fs, order):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    # padlen을 조정하거나 제거. 예: data의 길이의 절반 또는 None으로 설정
    padlen = min(len(data)//2, 2)  # 데이터 길이의 절반과 2 중 작은 값을 선택
    y = filtfilt(b, a, data, padlen=padlen)
    return y

# 필터링된 가속도 데이터 예
acc_mag = np.sqrt(accX**2 + accY**2 + accZ**2)
print("Data length:", len(acc_mag))
acc_mag_filt = low_pass_filter(acc_mag, cutoff=5, fs=1/sample_period, order=1)

# 그래프 표시
plt.figure(figsize=(12, 6))
plt.plot(time[index_sel], acc_mag_filt, label='Filtered Acceleration Magnitude')
plt.title('Filtered Acceleration')
plt.xlabel('Time [s]')
plt.ylabel('Acceleration [g]')
plt.legend()
plt.show()


Data length: 0


ValueError: The length of the input vector x must be greater than padlen, which is 0.

In [7]:
import os
print(os.getcwd())

d:\20192230 이동섭\IMU sensor
